In [17]:
# Settings
#filename    = '/home/axel/PycharmProjects/pax/tpc_kr_150410_8k.hdf5'
filename    = '/home/a_fieg02/Desktop/Data/MsTPC/Kr_1000TH.hdf5'
file_format = 'hfd5'
#loglevel    = 'DEBUG'

In [18]:
import NotebookLoader
import Function_definitions as fn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', size=16)
import h5py
import units
from recarray_tools import append_fields,fields_view, group_by, filter_on_fields, fields_data

# Optional progress bar
try:
    from tqdm import tqdm
except ImportError:
    def dummy(*args,**kwargs):
        return args[0]
    tqdm = dummy

In [19]:
# Slurp peaks and events into memory
# WARNING: For a large dataset, use pax to remove big low-level fields first
# (area_per_channel, does_channel_contribute, does_channel_have_noise)
import h5py
f = h5py.File(filename)
events = f.get('Event')[:]
peaks = f.get('Peak')[:]
f.keys()
f.close()
if len(events) == 0:
    raise ValueError("You don't have any events in this dataset!")

In [4]:
# Group peaks by event
peaks_per_event = group_by(peaks, 'Event')
assert len(peaks_per_event) == len(events)

peak_types = [b's1', b's2', b'unknown', b'noise', b'lone_pulse']
# Add number of individual peak types
for pt in peak_types:
    events = append_fields(events,
                           'n_'+pt.decode(),
                           np.array([len(x[x['type'] == pt]) for x in peaks_per_event]))
    
print('done')
print("Tell me the peaktypes S2:", len(peaks[(peaks['type'] == b's2')])) 
print("Tell me the peaktypes S1:", len(peaks[(peaks['type'] == b's1')])) 
print("Tell me the peaktypes noise:", len(peaks[(peaks['type'] == b'noise')])) 
print("Tell me the peaktypes unknown:", len(peaks[(peaks['type'] == b'unknown')]))
print("Tell me the peaktypes lone_pulse:", len(peaks[(peaks['type'] == b'lone_pulse')]))

done
Tell me the peaktypes S2: 82
Tell me the peaktypes S1: 238436
Tell me the peaktypes noise: 0
Tell me the peaktypes unknown: 1374
Tell me the peaktypes lone_pulse: 0


3

In [15]:
def plot_density(selected, xvar, yvar, xmin, xmax, ymin, ymax, log=False):
    
    for key in (xvar, yvar):
        try:
            labels[key]
        except KeyError:
            labels[key] = key

    xbins = np.linspace(xmin, xmax, 100)
    ybins = np.linspace(ymin, ymax, 100)

    counts, _, _ = np.histogram2d(selected[xvar], selected[yvar], bins=(xbins, ybins))
    counts = counts.T

    fig, ax = plt.subplots(figsize=(12, 7))
    
    norm=None
    if log:
        norm=LogNorm()
    
    a = ax.pcolormesh(xbins, ybins, counts, norm=norm)

    cbar = plt.colorbar(a)
    cbar.ax.get_yaxis().labelpad = 25
    cbar.set_label('counts', rotation=270)
    plt.title(dataset)
    plt.xlabel(labels[xvar])
    plt.ylabel(labels[yvar])

    plt.show()
    return
